<a href="https://colab.research.google.com/github/areeshaj3109/Scrapes/blob/main/LuLu-Scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cloudscraper
!pip install selenium
!apt update
!apt install chromium-chromedriver


from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import cloudscraper
import requests
from bs4 import BeautifulSoup

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

scraper = cloudscraper.create_scraper()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.

In [ ]:
def get_product_info(url):
    name = ''
    price = ''
    desc = ''
    img = ''
    domain = 'https://www.luluhypermarket.com'

    response = scraper.get(url)

    if response.status_code == 200:

        response = response.content
        soup = BeautifulSoup(response, 'html.parser')

        name_tag = soup.find('h1', class_ = 'product-name')
        if name_tag:
            name = name_tag.text.strip()
        #print(name)

        price_tag = soup.find('span', class_ = 'item price')
        if price_tag:
            price = price_tag.text.strip()[4:]
        #print(price)

        desc_tag = soup.find('div', class_ = 'description-block mb-3 mt-md-0')
        if desc_tag:
            desc = desc_tag.text.replace('More Information >', '').replace('Product Summary', '').strip()
        #print(desc)

        img_tag = soup.find('div', class_ = 'larg-img-container')
        if img_tag:
            img = img_tag.find('img')['src']
            img = domain + img
        #print(img)

        return {
            'Name': name,
            'Price': price,
            'Description': desc,
            'Image Link': img
        }

In [ ]:
def get_prod_urls(url):

    driver.get(url)

    SCROLL_PAUSE_TIME = 2

    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    prod_url_lst = []
    domain = 'https://www.luluhypermarket.com'
    prod_url = ''

    prod_tag_lst = soup.find_all('div', class_ = 'product-box js-gtmGA4-selct_item prevent-child-click')

    for tag in prod_tag_lst:
        prod_url = tag.get('data-url')
        prod_url = domain + prod_url
        prod_url_lst.append(prod_url)

    return prod_url_lst

In [ ]:
def get_subcat_urls(url):

    cat = ''
    subcat = ''
    subcat_url = ''
    domain = 'https://www.luluhypermarket.com'
    lst = []

    response = scraper.get(url)

    if response.status_code == 200:
        response = response.content
        soup = BeautifulSoup(response, 'html.parser')

    cat_menu_tag = soup.find_all('li', class_ = 'first-level with-dropdown')[4]

    subcat_tag_lst = cat_menu_tag.find_all('div', class_ = 'col-2')

    for tag in subcat_tag_lst:
        title_tag = tag.find('div', class_ = 'container-col-title')
        cat = title_tag.text.strip()

        links_tag = tag.find('div', class_ = 'container-col-links')
        subcat_link_tag_lst = links_tag.find_all('a')

        for tag in subcat_link_tag_lst:
            subcat = tag.text.strip()
            subcat_url = domain + tag.get('href')

            lst.append({
                'Category': cat,
                'Subcategory': subcat,
                'Subcategory url': subcat_url
            })

    return lst

In [ ]:
url = 'https://www.luluhypermarket.com/en-qa'

In [ ]:
subcat_lst = get_subcat_urls(url)
subcat_lst

[{'Category': 'Mobiles',
  'Subcategory': 'Smart Phones',
  'Subcategory url': 'https://www.luluhypermarket.com/en-qa/mobiles-smart-phones/c/HY00214730'},
 {'Category': 'Mobiles',
  'Subcategory': 'Featured Phones',
  'Subcategory url': 'https://www.luluhypermarket.com/en-qa/mobiles-featured-phones/c/HY00214734'},
 {'Category': 'Mobile Accessories',
  'Subcategory': 'Power Adapters & Chargers',
  'Subcategory url': 'https://www.luluhypermarket.com/en-qa/mobiles-mobile-accessories-power-adapters-chargers/c/HY00214720'},
 {'Category': 'Mobile Accessories',
  'Subcategory': 'Mobile Cases',
  'Subcategory url': 'https://www.luluhypermarket.com/en-qa/mobiles-mobile-accessories-cases/c/HY00214721'},
 {'Category': 'Mobile Accessories',
  'Subcategory': 'Mobile Hands Free',
  'Subcategory url': 'https://www.luluhypermarket.com/en-qa/mobiles-mobile-accessories-hands-free/c/HY00214722'},
 {'Category': 'Mobile Accessories',
  'Subcategory': 'Utility Cables',
  'Subcategory url': 'https://www.lulu

In [ ]:
for subcat_dict in subcat_lst:
    url = subcat_dict['Subcategory url']
    lst = get_prod_urls(url)
    subcat_dict['Product urls'] = lst
    print(subcat_dict['Subcategory'], len(lst))

Smart Phones 432
Featured Phones 25
Power Adapters & Chargers 118
Mobile Cases 172
Mobile Hands Free 332
Utility Cables 114
Power Banks 81
Screen Protectors 48
Mobile Car Accessories 46
Wireless Mini Speakers 24
Other Mobile Accessories 37
Smart Watches 266
Fitness Trackers 34
Wearable Accessories 4


In [ ]:
subcat_lst

[{'Category': 'Mobiles',
  'Subcategory': 'Smart Phones',
  'Subcategory url': 'https://www.luluhypermarket.com/en-qa/mobiles-smart-phones/c/HY00214730',
  'Product urls': ['https://www.luluhypermarket.com/en-qa/oppo-reno10pro-5g-smartphone-12gb-ram-256gb/p/2139268',
   'https://www.luluhypermarket.com/en-qa/samsung-galaxy-s23-ultra-dual-sim-5g-smartphone-12-gb/p/2075171',
   'https://www.luluhypermarket.com/en-qa/samsung-galaxy-s23-ultra-dual-sim-5g-smartphone-12-gb/p/2075167',
   'https://www.luluhypermarket.com/en-qa/samsung-galaxy-s23-ultra-dual-sim-5g-smartphone-12-gb/p/2275289',
   'https://www.luluhypermarket.com/en-qa/samsung-galaxy-s23-ultra-dual-sim-5g-smartphone-12-gb/p/2275288',
   'https://www.luluhypermarket.com/en-qa/samsung-galaxy-s23-ultra-dual-sim-5g-smartphone-12-gb/p/2075170',
   'https://www.luluhypermarket.com/en-qa/samsung-galaxy-s23-ultra-dual-sim-5g-smartphone-12-gb/p/2075168',
   'https://www.luluhypermarket.com/en-qa/nothing-phone-1-dual-sim-5g-smartphone-8-g

In [ ]:
final_lst = []

for subcat_dict in subcat_lst:

    lst = subcat_dict['Product urls']
    c = 1

    for url in lst:

        print(f"{subcat_dict['Subcategory']} Progress: {c}/{len(lst)}")
        c += 1

        prod_info = get_product_info(url)
        prod_info['Category'] = subcat_dict['Category']
        prod_info['Subcategory'] = subcat_dict['Subcategory']

        final_lst.append(prod_info)

Smart Phones Progress: 1/432
Smart Phones Progress: 2/432
Smart Phones Progress: 3/432
Smart Phones Progress: 4/432
Smart Phones Progress: 5/432
Smart Phones Progress: 6/432
Smart Phones Progress: 7/432
Smart Phones Progress: 8/432
Smart Phones Progress: 9/432
Smart Phones Progress: 10/432
Smart Phones Progress: 11/432
Smart Phones Progress: 12/432
Smart Phones Progress: 13/432
Smart Phones Progress: 14/432
Smart Phones Progress: 15/432
Smart Phones Progress: 16/432
Smart Phones Progress: 17/432
Smart Phones Progress: 18/432
Smart Phones Progress: 19/432
Smart Phones Progress: 20/432
Smart Phones Progress: 21/432
Smart Phones Progress: 22/432
Smart Phones Progress: 23/432
Smart Phones Progress: 24/432
Smart Phones Progress: 25/432
Smart Phones Progress: 26/432
Smart Phones Progress: 27/432
Smart Phones Progress: 28/432
Smart Phones Progress: 29/432
Smart Phones Progress: 30/432
Smart Phones Progress: 31/432
Smart Phones Progress: 32/432
Smart Phones Progress: 33/432
Smart Phones Progre